In [1]:
import swiftestio as swio
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def makemovie(ds):
    radscale = 20
    RMars = 3389500.0
    # set up the figure
    fig = plt.figure(figsize=(16, 9))
    canvas_width, canvas_height = fig.canvas.get_width_height()

    def data_stream(frame):
        while True:
            d = ds.isel(time=frame)
            ds['mass'] = ds['mass'] / config['GU']
            ds['radmarker'] = ds['radius'].fillna(0)
            ds['radmarker'] = ds['radmarker'] / ds['radmarker'].max()
            mass = d['mass'].values
            radius = d['radius'].values / radscale
            a = d['a'].values / RMars
            e = d['e'].values
            name = d['id'].values
            npl = d['npl'].values
            radmarker = d['radmarker'] * radscale
            t = ds.coords['time'].values[frame]
            yield t, name, mass, radius, npl, np.c_[a, e], radmarker

    def update(frame):
        """Update the scatter plot."""
        t, name, mass, radius, npl, pl, radmarker = next(data_stream(frame))

        title.set_text(f'Time = ${t / 24 / 3600:4.1f}$ days with ${npl:4.0f}$ particles')

        # We need to return the updated artist for FuncAnimation to draw..
        # Note that it expects a sequence of artists, thus the trailing comma.
        scat.set_sizes(radmarker)
        scat.set_offsets(pl)
        scat.set_facecolor('red')

        return scat, title,

    # First frame
    """Initial drawing of the scatter plot."""
    t, name, mass, radius, npl, pl, radmarker = next(data_stream(0))

    ax = plt.axes(xlim=(xmin, xmax), ylim=(ymin, ymax))
    ax.margins(x=10, y=1)
    ax.set_xlabel('Semi Major Axis ($R_{Mars}$)', fontsize='16', labelpad=1)
    ax.set_ylabel('Eccentricity', fontsize='16', labelpad=1)
    ax.set_yscale('log')

    title = ax.text(0.50, 1.05, "", bbox={'facecolor': 'w', 'alpha': 0.5, 'pad': 5}, transform=ax.transAxes,
                    ha="center")

    title.set_text(f'Time = ${t / 24 / 3600:4.1f}$ days with ${npl:f}$ particles')
    scat = ax.scatter(pl[:, 0], pl[:, 1], marker='o', s=radmarker, c='red', alpha=0.25)  # , vmin = 1000, vmax = 3000)

    # Open an ffmpeg process
    outf = 'aescatter.mp4'
    cmdstring = ('ffmpeg',
                 '-y', '-r', '1',  # overwrite, 1fps
                 '-s', '%dx%d' % (canvas_width, canvas_height),  # size of image string
                 '-pix_fmt', 'argb',  # format
                 '-f', 'rawvideo', '-i', '-',  # tell ffmpeg to expect raw video from the pipe
                 '-vcodec', 'mpeg4', outf)  # output encoding
    p = subprocess.Popen(cmdstring, stdin=subprocess.PIPE)

    # Draw frames and write to the pipe
    for frame in range(nframes):
        print(f'Generating frame {frame} of {nframes}')
        # draw the frame
        scat, title, = update(frame)
        fig.canvas.draw()

        # extract the image as an ARGB string
        string = fig.canvas.tostring_argb()

        # write to pipe
        p.stdin.write(string)

    # Finish up
    p.communicate()

In [3]:
config = swio.read_swiftest_config("param.in")

Reading Swiftest file param.in


In [4]:
marsdisk = swio.swiftest2xr(config)

Reading in time 3500.0
Creating Dataset
Successfully converted 8 output frames.


In [5]:
marsdisk['time'].size

8

In [6]:
makemovie(marsdisk)

KeyError: 'radmarker'

<Figure size 1152x648 with 0 Axes>